In [350]:
try:
    from tqdm import tqdm
except ImportError:
    def tqdm(iterator, *args, **kwargs):
        return iterator
    
import numpy as np
import scipy as sp
from scipy import constants
from pylab import *
import joblib


import os,sys
from importlib import reload
import copy
from collections import namedtuple

sys.path.append("../tracker")

In [539]:
import kalmanfilter as KF
import utilities as Util
import datatypes
import vertexfinder as VF

reload(VF)
reload(Util)

<module 'utilities' from '../tracker\\utilities.py'>

In [540]:
tracks=joblib.load("tracks_example.joblib")
# tracks

In [541]:
vf = VF.VertexFitter(debug=True)
vf.parameters["cut_vertex_SeedDist"]=400
vf.parameters["cut_vertex_SeedChi2"]=40000
vf.parameters["cut_vertex_TrackAddDist"]=400
vf.parameters["cut_vertex_TrackAddChi2"]=30
vf.parameters["cut_vertex_VertexChi2ReducedAdd"]=10
vf.parameters["cut_vertex_TrackDropChi2"]=10
vf.parameters["cut_vertex_VertexChi2Reduced"]=5
vertices = vf.run(tracks)
# %timeit -n 2 vf.run(tracks)

VertexSeed(x0=1938.3960598398905, y0=9737.391326829067, z0=9473.245814560687, t0=31.435021915324477, cov=0, chi2=767.5219051045467, dist=126.96329602115777, Ntracks=0, trackind1=0, trackind2=1, score=2472.514336065329)
--- New seed for vertex --- 
  Seed VertexSeed(x0=1938.3960598398905, y0=9737.391326829067, z0=9473.245814560687, t0=31.435021915324477, cov=0, chi2=767.5219051045467, dist=126.96329602115777, Ntracks=0, trackind1=0, trackind2=1, score=2472.514336065329)
 -> Track [3] added to vertex. Vertex chi2_r 1.98; vertex chi2 increment 9.76. Track: (1322.9312243791464, 9893.0, 10240.743959423311, 89.67492509000257, -0.953591017019463, 1, 1.084589776465581, 0.036783691065393684)
 -> Track [2] added to vertex. Vertex chi2_r 1.51; vertex chi2 increment 2.23. Track: (1238.3433915583182, 9893.0, 10006.519410496881, 59.2810215983687, -0.7367524891924243, 1, 1.170478487543455, 0.025723260543494206)
Vertex found! track indices: [0, 1, 3, 2]
  Tracks to vertex chi2: [0.25043280530758927, 0

In [502]:
0.01*(7936-7000-1950), 0.01*(-232.8637214423652), 0.01*(9353.048555922185-8547) 

(-10.14, -2.3286372144236522, 8.060485559221853)

In [69]:
print(len(vertices))
print(len(vertices[0].tracks))
print(vertices[0])

0


IndexError: list index out of range

In [ ]:
def root_hits_extractor_tracks(Tree, entry):
    Tree.GetEntry(entry)
    c2list = lambda x: np.array([x.at(i) for i in range(x.size())])

    Digi_layer = c2list(Tree.Digi_layer_id)
    # Only select hits in tracking layer
    mask = (Digi_layer>1) & (Digi_layer<=5) 

    Digi_x = c2list(Tree.Digi_x)[mask]
    Digi_y = c2list(Tree.Digi_y)[mask]
    Digi_z = c2list(Tree.Digi_z)[mask]
    Digi_t = c2list(Tree.Digi_time)[mask]
    Digi_track_id = c2list(Tree.Digi_track_id)[mask]
    Digi_layer = Digi_layer[mask]
    Hits = Util.hit.make_hits(Digi_x, Digi_y, Digi_z, Digi_t, Digi_layer)

    track_ids = np.unique(Digi_track_id)
    track_hits  = {}
    for trackid in track_ids:
        mask = Digi_track_id==trackid
        hit_group = []
        for i in np.flatnonzero(mask):
            hit_group.append(Hits[i])
        track_hits[int(trackid)] = hit_group

    return track_hits